## S3_series_feature.py

In [1]:
import warnings; warnings.simplefilter("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc, os, random
import time, datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from utils import *
root = args.root
seed = args.seed


available gpus: [0]


In [2]:
train = pd.read_feather("./input/train_denoise.feather")
test = pd.read_feather("./input/test_denoise.feather")

train_y = pd.read_csv(f"/mnt/sdb/KAGGLE_DATA/amex-default-prediction/train_labels.csv")

In [3]:
## ## debug --
## train = train.head(10000)
## test = test.head(10000)
## train_y = train_y.head(10000)

In [4]:
train = train.merge(train_y, how="left", on=id_name)

In [5]:
def one_hot_encoding(df, cols, is_drop=True):
    for col in cols:
        print("one hot encoding:", col)
        dummies = pd.get_dummies(pd.Series(df[col]), prefix="oneHot_%s"%col)
        df = pd.concat([df, dummies], axis=1)
    if is_drop:
        df.drop(cols, axis=1, inplace=True)
    return df

In [6]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
eps = 1e-3

In [7]:
print(train.shape, test.shape)

(5531451, 191) (11363762, 190)


In [8]:
lgb_config = {
    "lgb_params": {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting": "dart",
        "max_depth": -1,
        "num_leaves": 64,
        "learning_rate": 0.035,
        "bagging_freq": 5,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.7,
        "min_data_in_leaf": 256,
        "max_bin": 63,
        #"min_sum_hessian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        "lambda_l1": 0.1,
        "lambda_l2": 30,
        "num_threads": 16,
        "verbosity": 1,
    },
    "feature_name": [col for col in train.columns if col not in [id_name, label_name, "S_2"]],
    "rounds": 4500,
    "early_stopping_rounds": 100,
    "verbose_eval": 50,
    "folds": 5,
    "seed": seed
}

In [9]:
Lgb_train_and_predict(train, test, lgb_config, gkf=True, aug=None, run_id="LGB_with_series_feature")

cp: cannot stat './*.sh': No such file or directory


[LightGBM] [Info] Number of positive: 1102596, number of negative: 3323080
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.574514 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6733
[LightGBM] [Info] Number of data points in the train set: 4425676, number of used features: 188
[50]	training's binary_logloss: 0.424557	valid_1's binary_logloss: 0.423357
[100]	training's binary_logloss: 0.37102	valid_1's binary_logloss: 0.369618
[150]	training's binary_logloss: 0.348986	valid_1's binary_logloss: 0.34755
[200]	training's binary_logloss: 0.338116	valid_1's binary_logloss: 0.336706
[250]	training's binary_logloss: 0.314579	valid_1's binary_logloss: 0.313154
[300]	training's binary_logloss: 0.302556	valid_1's binary_logloss: 0.301218
[350]	training's binary_logloss: 0.294663	valid_1's binary_logloss: 0.293401
[400]	training's binary_loglos

(                                               customer_ID    target
 0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001787
 1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001801
 2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.002086
 3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001897
 4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001420
 ...                                                    ...       ...
 5531446  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.003283
 5531447  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002004
 5531448  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002349
 5531449  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.001699
 5531450  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002380
 
 [5531451 rows x 2 columns],
                                                 customer_ID  prediction
 0         00000469ba478561f23a92a868bd366de6f6527a684c9